## Demo5
In this notebook, a whole demo will be presented.

In [12]:
import jieba
import re
import chardet
from gensim.models import KeyedVectors
import numpy as np
import pandas as pd
import math
import datetime
import prettytable as pt

## Load File

In [13]:
def loadSchool():
    path1 = 'data/chinese_university_list.csv'
    school_df = pd.read_csv(path1, header=None, delimiter=",", skiprows=4, names=["rank", "name", "code", "department", "city", "level", "notes"])
    school_df = pd.DataFrame(school_df, columns=['name'])
    
    path2 = 'data/all_university_list.csv'
    school_global_df = pd.read_csv(path2, header=None, delimiter=',', skiprows=2, names=['Name_en', 'name', 'rank', 'score', 'location'])
    school_global_df = pd.DataFrame(school_global_df, columns=['name'])
    
    school_df = pd.concat([school_df, school_global_df], axis=0, ignore_index=True)
    return school_df

In [14]:
def loadDegree():
    degree = {'name': ['本科', '硕士' ,'研究生', '博士']}
    degree_df = pd.DataFrame(degree)
    return degree_df

In [15]:
def loadCompany():
    path1 = 'data/company_list_ch.csv'
    company_ch_df = pd.read_csv(path1, header=None, delimiter=",", skiprows=2, names=['rank', 'name', 'Location', 'Income'])
    company_ch_df = pd.DataFrame(company_ch_df, columns=['name'])
    path2 = 'data/member-data.csv'
    company_df = pd.read_csv(path2, header=None, delimiter=",", skiprows=2, names=['name', 'No.', 'Resume', 'Position'])
    company_df = pd.DataFrame(company_df, columns=['name'])
    company_ch_df = pd.concat([company_ch_df, company_df], axis=0, ignore_index=True)
    
    print(f'before dedup, company size: {company_ch_df.shape[0]}')
    company_ch_df = company_ch_df.drop_duplicates(subset=['name'], keep='first')
    print(f'after dedup, company size: {company_ch_df.shape[0]}')
    return company_ch_df

In [16]:
def loadPosition():
    path5 = 'data/position.csv'
    position_df1 = pd.read_csv(path5, header=None, delimiter=",", skiprows=1, names=['name'])
    position_df1 = pd.DataFrame(position_df1)

    path6 = 'data/member-data.csv'
    position_df2 = pd.read_csv(path6, header=None, delimiter=",", skiprows=1, names=['Company', 'No.', 'Resume', 'name'])
    position_df2 = pd.DataFrame(position_df2, columns=['name'])

    member_position_list = []
    # process position in member-data
    for index, row in position_df2.iterrows():
        position = row['name']
        if isinstance(position, float) or position == " " or position.isalpha():
            continue
        if "&" in position:
            temp1 = position.split('&')
            member_position_list += temp1
        elif " " in position:
            temp2 = position.split( )
            member_position_list += temp2
        else:
            member_position_list.append(position)

    position_df3 = pd.DataFrame(member_position_list, columns=['name'])
    position_df1 = pd.concat([position_df1, position_df3], axis=0, ignore_index=True)
    print(f'before dedup, position size: {position_df1.shape[0]}')
    position_df1 = position_df1.drop_duplicates(subset=['name'], keep='first')
    print(f'after dedup, position size: {position_df1.shape[0]}')
    # print(df1.head())
    return position_df1

In [39]:
def loadMember():
    path = 'data/member-data.csv'
    member_df = pd.read_csv(path, header=None, delimiter=",", skiprows=1, names=['Company', 'No.', 'Resume', 'Position'])
    member_df = pd.DataFrame(member_df, columns=['Resume'])
    return member_df

## Preprocess Text

In [18]:
def removeStopWords(seglist):
    stopwords = {}
    fstop = open('data/stopwords_cn.txt', 'r', encoding='utf-8', errors='ignore')
    for w in fstop:
        stopwords[w.strip()] = w.strip()

    fstop.close()
    stopwords[' '] = ' '
    
    segListSanitized = []

    for word in seglist:
        # translation
        if word == 'omnigo':
            word = '酷刻'
        if word == 'Aibee':
            word = '爱笔'
        if word == 'ilife':
            word = '爱乐福'
        if word == 'oracleen':
            word = '爱芽'
        if word not in stopwords:
            segListSanitized.append(word)
    return segListSanitized

In [19]:
def preprocess(text):
    # remove punctuations
    text = re.sub(r"[\s+\.\!\/_,$%^*()?;；:【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", text)
    text = text.lower()
    # seperate words
    words = jieba.cut(text, cut_all=False)
    seglist = list(words)
    # remove stopwords
    segListSanitized = removeStopWords(seglist)
    print(f'Before sanitize, len: {len(seglist)}. After sanitize, len: {len(segListSanitized)}')

    return segListSanitized

## N-gram Algorithm

In [20]:
def getNgrams(wordList, n):
    '''
    This function only generete N-Grams
    '''
    output = []
    for i in range (len(wordList) - n + 1):
        n_gram_temp = "".join(wordList[i:i+n])
        output.append(n_gram_temp)
    return output

In [21]:
def generateNgrams(wordList, n):
    '''
    This function genereates [1, N]-Grams
    '''
    result = set()
    for i in range(n):
        temp = getNgrams(wordList, i+1)
        result = result | temp
    
    return result

In [22]:
def generateNgramsV2(wordList, n):
    '''
    This function genereates [1, N]-Grams
    '''
    result = []
    for i in range(n):
        temp_list = getNgrams(wordList, i+1)
        temp = list(set(temp_list))
        temp.sort(key=temp_list.index)
        result.append(temp)
        
    return result

## Word Embedding

In [23]:
model = KeyedVectors.load('./test_50.bin')

In [24]:
def calculate_cosine_similarity(a, b):
    vector_a = np.mat(a)
    vector_b = np.mat(b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    
    if denom == 0:
        return 0.0
    
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [25]:
def calculate_IDF(df):
    company_num = 0
    m = dict()
    for index, row in df.iterrows():
        name = row['name']
        name = re.sub(r"[\s+\.\!\/_,$%^*()?;；:【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", name)
        name = name.lower()
        if isinstance(name, float) or name == " ":
            continue
        company_num += 1
        words = jieba.cut(name, cut_all=False)
        word_list = list(words)
        for word in word_list:
            if word in m.keys():
                m[word] +=1
            else:
                m[word] = 1
    
    idf = dict()
    
    for (k, v) in m.items():
        idf[k] = math.log(((1+company_num) / v), 10)
    
    # Normalize min-max
    v = idf.values()
    max_value = max(v)
    min_value = min(v)
    denom = max_value - min_value
    for (k, v) in idf.items():
        idf[k] = (idf[k] - min_value) / denom

    return idf

In [26]:
def generateEmbeddings(name):
    words = jieba.cut(name, cut_all=False)
    word_list = list(words)
    v = np.zeros((200))
    for word in word_list:
        if word in model.vocab:
            v += model[word]
    
    v /= len(v)
    return v

In [27]:
def generateEmbeddingsWithIDF(name, idf):
    words = jieba.cut(name, cut_all=False)
    word_list = list(words)
    v = np.zeros((200))
    for word in word_list:
        if word in model.vocab:
            v += model[word] * idf[word]
    
    v /= len(v)
    return v

In [28]:
def preprocess_entity_list(df, model):
    '''
    df: dafaframe
    model: word embedding model
    '''
    
    df['embeddings'] = ''
    for index, row in df.iterrows():
        # df.loc[index, 'embeddings'] = z
        name = row['name']
        if isinstance(name, float):
            continue
        name = name.lower()
        if name in model.vocab:
            vec = model[name]
        else:
            vec = generateEmbeddings(name)
        df.set_value(index, 'embeddings', vec)

    return df

In [29]:
def preprocess_entity_list_withIDF(df, model, idf):
    '''
    df: dafaframe
    model: word embedding model
    idf: IDF for each word
    '''
    
    df['embeddings_idf'] = ''
    for index, row in df.iterrows():
        name = row['name']
        name = re.sub(r"[\s+\.\!\/_,$%^*()?;；:【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", name)
        name = name.lower()
        if isinstance(name, float):
            continue
        if name in model.vocab:
            vec = model[name]
        else:
            vec = generateEmbeddingsWithIDF(name, idf)
        df.set_value(index, 'embeddings_idf', vec)

    return df

## Entity Link

In [30]:
def linkEntity(output, model, company_df, position_df, school_df, degree_df, company_threshold1, company_threshold2, position_threshold, school_threshold, degree_threshold, f):
    company_entity = []
    position_entity = []
    school_entity = []
    degree_entity = []
    for index, li in enumerate(output):
        print(f'process {index}-Gram')
        print(f'process {index}-Gram', file=f)
        for term in li:
            if len(term) <= 1:
                continue
            if term in model.vocab:
                term_vec = model[term]

                # Link Company
                company_candidate = dict()
                for index, row in company_df.iterrows():
                    name = row['name']
                    if isinstance(name, float):
                        continue
                    name_vec = row['embeddings_idf']
                    sim = calculate_cosine_similarity(term_vec, name_vec)
                    if (sim > company_threshold1):
                        company_candidate[name] = sim
                if len(company_candidate) != 0:
                    company_candidate = sorted(company_candidate.items(), key=lambda item:item[1], reverse=True)
                    print(f'company entity found: {term}->{company_candidate[0][0]}, sim = {company_candidate[0][1]}')
                    print(f'company entity found: {term}->{company_candidate[0][0]}, sim = {company_candidate[0][1]}', file=f)
                    company_entity.append(company_candidate[0])
                    
                # Link Position
                position_candidate = dict()
                for index, row in position_df.iterrows():
                    name = row['name']
                    name_vec = row['embeddings']
                    sim = calculate_cosine_similarity(term_vec, name_vec)
                    if (sim > position_threshold):
                        position_candidate[name] = sim
                if len(position_candidate) != 0:
                    position_candidate = sorted(position_candidate.items(), key=lambda item:item[1], reverse=True)
                    print(f'position entity found: {term}->{position_candidate[0][0]}, sim = {position_candidate[0][1]}')
                    print(f'position entity found: {term}->{position_candidate[0][0]}, sim = {position_candidate[0][1]}', file=f)
                    position_entity.append(position_candidate[0])
                
                # Link School
                school_candidate = dict()
                for index, row in school_df.iterrows():
                    name = row['name']
                    if isinstance(name, float):
                        continue
                    name_vec = row['embeddings']
                    sim = calculate_cosine_similarity(term_vec, name_vec)
                    if (sim > school_threshold):
                        school_candidate[name] = sim
                if len(school_candidate) != 0:
                    school_candidate = sorted(school_candidate.items(), key=lambda item:item[1], reverse=True)
                    print(f'university entity found: {term}->{school_candidate[0][0]}, sim = {school_candidate[0][1]}')
                    print(f'university entity found: {term}->{school_candidate[0][0]}, sim = {school_candidate[0][1]}', file=f)
                    school_entity.append(school_candidate[0])
                    
                # Link Degree
                degree_candidate = dict()
                for index, row in degree_df.iterrows():
                    name = row['name']
                    name_vec = row['embeddings']
                    sim = calculate_cosine_similarity(term_vec, name_vec)
                    if (sim > degree_threshold):
                        degree_candidate[name] = sim
                if len(degree_candidate) != 0:
                    degree_candidate = sorted(degree_candidate.items(), key=lambda item:item[1], reverse=True)
                    print(f'degree entity found: {term}->{degree_candidate[0][0]}, sim = {degree_candidate[0][1]}')
                    print(f'degree entity found: {term}->{degree_candidate[0][0]}, sim = {degree_candidate[0][1]}', file=f)
                    degree_entity.append(degree_candidate[0])
            else:
                term_vec = generateEmbeddings(term)
                for index, row in company_df.iterrows():
                    name = row['name']
                    if isinstance(name, float):
                        continue
                    name_vec = row['embeddings']
                    sim = calculate_cosine_similarity(term_vec, name_vec)
                    if (sim == 0.0 or sim > company_threshold2):
                        is_match = exact_match(name, term)
                        if is_match:
                            print(f'company entity found by exact match: {term}->{name}')
                            print(f'company entity found by exact match: {term}->{name}', file=f)
                            company_entity.append([name, 1.0])
                            break
                
    return company_entity, position_entity, school_entity, degree_entity

In [31]:
def exact_match(entity, term):
    keyword = extractKeyword(term)
    is_match = False
    for key in keyword:
        idx = entity.find(key)
        if idx != -1:
            is_match = True
    return is_match

In [32]:
def extractKeyword(term):
    '''
    Extract keyword in a term, whose embedding can not be found in vocabulary
    Return a list
    '''
    words = jieba.cut(term, cut_all=False)
    word_list = list(words)
    if len(word_list) > 1:
        word_list = getNgrams(word_list, 2)
    v = np.zeros((200))
    keyword = []
    for word in word_list:
        if word not in model.vocab:
            keyword.append(word)
    return keyword

In [48]:
def print_table_company_position(company_entity, position_entity, f):
    company_list = []
    company_sim_list = []
    position_list = []
    position_sim_list = []
        
    min_len = min(len(company_entity), len(position_entity))
    
    for i, s in enumerate(company_entity):
        if i == min_len:
            break
        company_list.append(s[0])
        company_sim_list.append(s[1])

    for i, d in enumerate(position_entity):
        if i == min_len:
            break
        position_list.append(d[0])
        position_sim_list.append(d[1])

    tb = pt.PrettyTable()
    tb.add_column("Company", company_list)
    tb.add_column("Company_Similarity", company_sim_list)
    tb.add_column("Position", position_list)
    tb.add_column("Position_Similarity", position_sim_list)
    print(tb)
    print(tb, file=f)

In [57]:
def print_table_school_degree(school_entity, degree_entity,f ):
    school_list = []
    school_sim_list = []
    degree_list = []
    degree_sim_list = []

    if len(degree_entity) == 0:
        d = dict()
        d= ['本科', 1.0]
        degree_entity.append(d)
        
    min_len = min(len(school_entity), len(degree_entity))
    
    for i, s in enumerate(school_entity):
        if i == min_len:
            break
        school_list.append(s[0])
        school_sim_list.append(s[1])

    for i, d in enumerate(degree_entity):
        if i == min_len:
            break
        degree_list.append(d[0])
        degree_sim_list.append(d[1])

    tb = pt.PrettyTable()
    tb.add_column("School", school_list)
    tb.add_column("School_Similarity", school_sim_list)
    tb.add_column("Degree", degree_list)
    tb.add_column("Degree_Similarity", degree_sim_list)
    print(tb)
    print(tb, file=f)

In [58]:
def print_table(company_entity, position_entity, school_entity, degree_entity, f):
    print_table_company_position(company_entity, position_entity, f)
    print_table_school_degree(school_entity, degree_entity, f)

## Main Function

In [59]:
company_df = loadCompany()
position_df = loadPosition()
school_df = loadSchool()
degree_df = loadDegree()
member_df = loadMember()

before dedup, company size: 11716
after dedup, company size: 3571
before dedup, position size: 5983
after dedup, position size: 199


In [60]:
company_idf = calculate_IDF(company_df)
company_df = preprocess_entity_list(company_df, model)
company_df = preprocess_entity_list_withIDF(company_df, model, company_idf)
position_df = preprocess_entity_list(position_df, model)
school_df = preprocess_entity_list(school_df, model)
degree_df = preprocess_entity_list(degree_df, model)

c:\users\梁育诚\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\梁育诚\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


### Parameters

In [61]:
company_thres1 = 0.98
company_thres2 = 0.9
position_thres = 0.98
school_thres = 0.91
degree_thres = 0.91

## Case Test

In [62]:
text1 = '__团队成员#1__先生是公司创始人,也是中国最有影响力的商界领袖之一。1982年,__团队成员#1__先生于华南理工大学毕业,进入TCL的前身-TTK家庭电器有限公司。1985年,他担任新成立的TCL通讯设备公司总经理,创立了TCL品牌。2003年,__团队成员#1__担任TCL集团股份有限公司董事长兼CEO,随后TCL集团整体上市。在他的领导下,2004年TCL一举收购了法国汤姆逊全球彩电业务与阿尔卡特全球手机业务。目前TCL集团已经成为拥有6万名员工,业务遍及全球80多个国家和地区。2013年,TCL集团营业总收入超过855亿元,液晶电视全球销量1766万台,实际产量全球第三,品牌销售全球第三;TCL手机全球销量5520万台,行业排名全球第五。2012年__团队成员#1__被新华网评为“最具社会责任感企业家”;2011年荣获《中国企业家》“最具影响力的25位企业领袖”终身成就奖;2009年被评为“CCTV中国经济年度人物十年商业领袖”;2008年获改革开放30年经济人物称号;2004年被评为Fortune杂志“亚洲年度经济人物”、TIME杂志和CNN全球最具影响力的25名商界人士,同年法国总统希拉克向__团队成员#1__先生颁发了法国国家荣誉勋章。__团队成员#1__是中共第十六大代表,第十届、第十一届、第十二届全国人大代表。__团队成员#1__担任的社会职务包括:中国电子视像行业协会会长;中国国际商会副会长;全国工商联执行委员、广东省工商联(总商会)副主席。'

In [63]:
output_path1 = 'output1.txt'
segListSanitized = preprocess(text1)
output = generateNgramsV2(segListSanitized, 3)
f1 = open(output_path1, "w", encoding='utf-8')
company_entity, position_entity, school_entity, degree_entity = linkEntity(output, model, company_df, position_df, school_df, degree_df, company_thres1, company_thres2, position_thres, school_thres, degree_thres, f1)
print_table(company_entity, position_entity, school_entity, degree_entity, f1)
f1.close()
print()

Before sanitize, len: 325. After sanitize, len: 239
process 0-Gram
position entity found: 创始人->创始人, sim = 1.000000029802326
company entity found: 中国->尚科宁家（中国）科技有限公司, sim = 0.9905265283315191
university entity found: 华南理工大学->华南理工大学, sim = 1.000000029802326
position entity found: 总经理->总经理, sim = 1.0000000596046448
position entity found: 董事长->董事长, sim = 1.000000029802326
position entity found: ceo->CEO, sim = 1.0
process 1-Gram
position entity found: 公司创始人->公司创始人, sim = 1.0000000298023295
process 2-Gram
company entity found: tcl集团股份有限公司->TCL集团股份有限公司, sim = 1.0
+------------------------------+--------------------+----------+---------------------+
|           Company            | Company_Similarity | Position | Position_Similarity |
+------------------------------+--------------------+----------+---------------------+
| 尚科宁家（中国）科技有限公司 | 0.9905265283315191 |  创始人  |  1.000000029802326  |
|     TCL集团股份有限公司      |        1.0         |  总经理  |  1.0000000596046448 |
+----------------------------

In [64]:
text2 = '__团队成员#13__先生,现任深圳市华星光电技术有限公司高级副总裁.1955 年9 月生,硕士,韩国籍.1973 年至1981 年,韩国延世大学材料工程本科毕业;1991年至1995 年,韩国延世大学材料工程研究生毕业,获硕士学位;2003 年至2006年,McGill University Business 专业MBA 毕业,获硕士学位.1981 年至1999年,历任LG 半导体有限公司制程工程师、存储器制程发展部部长、高级技术中心(ATC)主管、C3 工厂厂长、执行总监;2000 年至2009 年,历任LG PHILIPS液晶显示IT 业务总部执行副总裁、LG PHILIPS 液晶显示生产技术中心总部执行副总裁;2009 年至2010 年,任日本FUHRMEISTER 电子高级顾问;2010 年3 月至今,任深圳市华星光电技术有限公司高级副总裁、总裁、首席执行官等职。'

In [67]:
output_path2 = 'output2.txt'
segListSanitized = preprocess(text2)
output = generateNgramsV2(segListSanitized, 3)
f2 = open(output_path2, "w", encoding='utf-8')
company_entity, position_entity, school_entity, degree_entity = linkEntity(output, model, company_df, position_df, school_df, degree_df, company_thres1, company_thres2, position_thres, school_thres, degree_thres, f2)
print_table(company_entity, position_entity, school_entity, degree_entity, f2)
f2.close()
print()

Before sanitize, len: 188. After sanitize, len: 124
process 0-Gram
position entity found: 副总裁->副总裁, sim = 1.000000029802326
degree entity found: 硕士->硕士, sim = 1.0
university entity found: 延世大学->延世大学, sim = 1.000000029802326
degree entity found: 本科毕业->本科, sim = 0.9349876046180725
degree entity found: 研究生->研究生, sim = 1.0
university entity found: mcgill->麦吉尔大学, sim = 0.9162431259445103
position entity found: 部长->部长, sim = 1.0000000596046448
position entity found: 总裁->总裁, sim = 1.000000029802326
process 1-Gram
position entity found: 高级副总裁->高级副总裁, sim = 1.0000000596046448
degree entity found: 研究生毕业->研究生, sim = 0.9151657819747925
company entity found by exact match: 半导体有限公司->山东华芯半导体有限公司
position entity found: 执行副总裁->执行副总裁, sim = 1.0000000596046448
position entity found: 首席执行官->首席执行官, sim = 1.0
process 2-Gram
company entity found by exact match: lg半导体有限公司->山东华芯半导体有限公司
company entity found by exact match: 半导体有限公司制程->山东华芯半导体有限公司
+------------------------+--------------------+----------+--------

In [69]:
text3 = '__团队成员#6__先生,1980年7月生,硕士研究生学历。2002年,福州大学经济学本科毕业;2006年,云南大学法律硕士研究生毕业。2006年8月至2014年2月,任职国泰君安证券股份有限公司,历任国泰君安证券香港公司财务顾问部高级经理、总经理,深圳总部机构客户部总监,从事香港与中国资本市场的投资银行业务。2014年3月加入TCL集团股份有限公司,任公司董事会办公室主任;2014年4月起任公司董事会秘书;2014年12月起任公司执委会成员;2015年4月起任TCL集团控股子公司全球播有限公司董事;2015年5月起任TCL通讯科技控股有限公司(02618.HK)非执行董事。'

In [70]:
output_path3 = 'output3.txt'
segListSanitized = preprocess(text3)
output = generateNgramsV2(segListSanitized, 3)
f3 = open(output_path3, "w", encoding='utf-8')
company_entity, position_entity, school_entity, degree_entity = linkEntity(output, model, company_df, position_df, school_df, degree_df, company_thres1, company_thres2, position_thres, school_thres, degree_thres, f3)
print_table(company_entity, position_entity, school_entity, degree_entity, f3)
f3.close()
print()

Before sanitize, len: 139. After sanitize, len: 111
process 0-Gram
degree entity found: 硕士->硕士, sim = 1.0
degree entity found: 研究生->研究生, sim = 1.0
university entity found: 福州大学->福州大学, sim = 1.0000000596046448
degree entity found: 本科毕业->本科, sim = 0.9349876046180725
university entity found: 云南大学->云南大学, sim = 1.0000000596046448
position entity found: 经理->经理, sim = 1.0000000596046448
position entity found: 总经理->总经理, sim = 1.0000000596046448
company entity found: 深圳->深圳坂云智行有限公司, sim = 1.0000000159812819
company entity found: 中国->尚科宁家（中国）科技有限公司, sim = 0.9905265283315191
position entity found: 董事->董事, sim = 1.0
company entity found: 科技->多玛凯拔科技有限公司, sim = 1.0000000094492707
process 1-Gram
degree entity found: 硕士研究生->硕士, sim = 0.9260654151439667
degree entity found: 研究生毕业->研究生, sim = 0.9151657819747925
position entity found: 高级经理->高级经理, sim = 1.0
position entity found: 董事会秘书->董事会秘书, sim = 1.0000000596046448
company entity found by exact match: 科技控股->盛世乐居（武汉）科技控股有限公司
position entity found: 执行董事-

In [143]:
text4 = '__团队成员#1__，北京爱国者新能源科技发展有限公司 CEO。'

In [147]:
segListSanitized = preprocess(text4)
output = generateNgramsV2(segListSanitized, 3)
company_entity, position_entity = linkCompanyAndPosition(output, model, company_df, position_df, 0.95, 0.9, 0.98)
print_table_company_position(company_entity, position_entity)
print()

Before sanitize, len: 16. After sanitize, len: 10
process 0-Gram
company entity found: 北京->宝希（北京）科技有限公司, sim = 0.9760900424845506
company entity found: 爱国者->爱国者电子科技有限公司, sim = 0.9713396702727501
company entity found: 新能源->上海烯美新能源科技有限公司, sim = 0.9637615946079356
company entity found: 科技->多玛凯拔科技有限公司, sim = 1.0000000094492707
company entity found: 发展->深圳市迈迪加科技发展有限公司, sim = 0.9604227663245726
position entity found: ceo->CEO, sim = 1.0
process 1-Gram
process 2-Gram
+--------------------------+--------------------+----------+---------------------+
|         Company          | Company_Similarity | Position | Position_Similarity |
+--------------------------+--------------------+----------+---------------------+
| 宝希（北京）科技有限公司 | 0.9760900424845506 |   CEO    |         1.0         |
+--------------------------+--------------------+----------+---------------------+



In [148]:
text5 = '__团队成员#1__，Omnigo机器人CEO。毕业于华中科技大学，原uArm创始团队核心成员，uArm机械臂主创设计师。'

In [150]:
segListSanitized = preprocess(text5)
output = generateNgramsV2(segListSanitized, 3)
company_entity, position_entity = linkCompanyAndPosition(output, model, company_df, position_df, 0.98, 0.9, 0.98)
print_table_company_position(company_entity, position_entity)
print()

Before sanitize, len: 27. After sanitize, len: 18
process 0-Gram
company entity found by exact match: 酷刻->广州酷刻科技有限公司
position entity found: ceo->CEO, sim = 1.0
process 1-Gram
process 2-Gram
+----------------------+--------------------+----------+---------------------+
|       Company        | Company_Similarity | Position | Position_Similarity |
+----------------------+--------------------+----------+---------------------+
| 广州酷刻科技有限公司 |        1.0         |   CEO    |         1.0         |
+----------------------+--------------------+----------+---------------------+



## Batch Process
Output to a file each 10 cases

In [75]:
begin = 51
finish = 100
f = open('output_batch2.txt', "w", encoding="utf-8")
for index, row in member_df.iterrows():
    if index < begin:
        continue
    if index > finish:
        break
    print(f'Handle No.{index} text')
    print(f'Handle No.{index} text', file=f)
    start = datetime.datetime.now()
    text = row['Resume']
    segListSanitized = preprocess(text)
    output = generateNgramsV2(segListSanitized, 3)
    company_entity, position_entity, school_entity, degree_entity = linkEntity(output, model, company_df, position_df, school_df, degree_df, company_thres1, company_thres2, position_thres, school_thres, degree_thres, f)
    print_table(company_entity, position_entity, school_entity, degree_entity, f)
    print()
    end = datetime.datetime.now()
    print(f'cost time: {end - start} sec')
    print(f'cost time: {end - start} sec', file=f)
    print('\n\n')
    print('\n\n', file=f)
    
f.close()

Handle No.51 text
Before sanitize, len: 11. After sanitize, len: 6
process 0-Gram
position entity found: 创始人->创始人, sim = 1.000000029802326
process 1-Gram
process 2-Gram
+---------+--------------------+----------+---------------------+
| Company | Company_Similarity | Position | Position_Similarity |
+---------+--------------------+----------+---------------------+
+---------+--------------------+----------+---------------------+
+--------+-------------------+--------+-------------------+
| School | School_Similarity | Degree | Degree_Similarity |
+--------+-------------------+--------+-------------------+
+--------+-------------------+--------+-------------------+

cost time: 0:00:14.195715 sec



Handle No.52 text
Before sanitize, len: 55. After sanitize, len: 42
process 0-Gram
university entity found: 清华大学->清华大学, sim = 1.0
degree entity found: 博士->博士, sim = 1.0
university entity found: 耶鲁大学->耶鲁大学, sim = 1.0
process 1-Gram
process 2-Gram
+---------+--------------------+----------+----

company entity found: 中国->尚科宁家（中国）科技有限公司, sim = 0.9905265283315191
position entity found: 副总经理->副总经理, sim = 1.0000000596046448
company entity found: 南京->南京控驰科技有限公司, sim = 0.9904987870454438
company entity found: 上海->馥德（上海）科技有限公司, sim = 0.9805150951364708
position entity found: 总经理->总经理, sim = 1.0000000596046448
position entity found: 副董事长->副董事长, sim = 1.0
position entity found: 董事->董事, sim = 1.0
process 1-Gram
process 2-Gram
+------------------------------+--------------------+----------+---------------------+
|           Company            | Company_Similarity | Position | Position_Similarity |
+------------------------------+--------------------+----------+---------------------+
| 尚科宁家（中国）科技有限公司 | 0.9905265283315191 | 副总经理 |  1.0000000596046448 |
|     南京控驰科技有限公司     | 0.9904987870454438 |  总经理  |  1.0000000596046448 |
|   馥德（上海）科技有限公司   | 0.9805150951364708 | 副董事长 |         1.0         |
+------------------------------+--------------------+----------+---------------------+
+--------

company entity found: 深圳->深圳坂云智行有限公司, sim = 1.0000000159812819
position entity found: 副总经理->副总经理, sim = 1.0000000596046448
process 1-Gram
company entity found by exact match: 系统有限公司->东莞市利发爱尔空气净化系统有限公司
process 2-Gram
company entity found by exact match: 自控系统有限公司->杭州罗莱迪思照明系统有限公司
company entity found by exact match: 系统有限公司营销中心->东莞市利发爱尔空气净化系统有限公司
+------------------------------+--------------------+----------+---------------------+
|           Company            | Company_Similarity | Position | Position_Similarity |
+------------------------------+--------------------+----------+---------------------+
| 尚科宁家（中国）科技有限公司 | 0.9905265283315191 | 副总经理 |  1.0000000596046448 |
+------------------------------+--------------------+----------+---------------------+
+--------+-------------------+--------+-------------------+
| School | School_Similarity | Degree | Degree_Similarity |
+--------+-------------------+--------+-------------------+
+--------+-------------------+--------+-------------------

process 2-Gram
company entity found by exact match: 企业集团有限责任->国家能源投资集团有限责任公司

company entity found by exact match: 集团有限责任公司->国家能源投资集团有限责任公司

+------------------------------+--------------------+------------+---------------------+
|           Company            | Company_Similarity |  Position  | Position_Similarity |
+------------------------------+--------------------+------------+---------------------+
| 尚科宁家（中国）科技有限公司 | 0.9905265283315191 |   总经理   |  1.0000000596046448 |
|   馥德（上海）科技有限公司   | 0.9805150951364708 | 监事会主席 |         1.0         |
+------------------------------+--------------------+------------+---------------------+
+--------+-------------------+--------+-------------------+
| School | School_Similarity | Degree | Degree_Similarity |
+--------+-------------------+--------+-------------------+
+--------+-------------------+--------+-------------------+

cost time: 0:03:15.025995 sec



Handle No.75 text
Before sanitize, len: 10. After sanitize, len: 5
process 0-Gram
pos

company entity found by exact match: 超清科技股份->安徽超清科技股份有限公司
process 2-Gram
company entity found by exact match: 安徽超清信息工程->安徽超清科技股份有限公司
company entity found by exact match: 现任安徽超清->安徽超清科技股份有限公司
company entity found by exact match: 安徽超清科技股份->安徽超清科技股份有限公司
company entity found by exact match: 超清科技股份有限公司->安徽超清科技股份有限公司
+------------------------------+--------------------+----------+---------------------+
|           Company            | Company_Similarity | Position | Position_Similarity |
+------------------------------+--------------------+----------+---------------------+
| 尚科宁家（中国）科技有限公司 | 0.9905265283315191 |   经理   |  1.0000000596046448 |
|    安徽爱依特科技有限公司    | 0.9904600772049361 |   董事   |         1.0         |
|     多玛凯拔科技有限公司     | 1.0000000094492707 | 采购经理 |         1.0         |
+------------------------------+--------------------+----------+---------------------+
+----------+-------------------+--------+-------------------+
|  School  | School_Similarity | Degree | Degree_Similarity

university entity found: 安徽大学->安徽大学, sim = 1.000000029802326
company entity found: 安徽->安徽爱依特科技有限公司, sim = 0.9904600772049361
company entity found: 科技->多玛凯拔科技有限公司, sim = 1.0000000094492707
position entity found: 经理->经理, sim = 1.0000000596046448
company entity found: 科技股份->华立捷科技股份有限公司, sim = 0.9999999984295311
position entity found: 董事->董事, sim = 1.0
process 1-Gram
company entity found by exact match: 科技有限责任->安徽大德中电科技有限责任公司
position entity found: 销售经理->销售经理, sim = 1.0
company entity found by exact match: 安徽超清->安徽超清科技股份有限公司
company entity found by exact match: 超清科技股份->安徽超清科技股份有限公司
process 2-Gram
company entity found by exact match: 文康科技有限责任->北京恒昊天科技有限责任公司
company entity found by exact match: 安徽超清信息工程->安徽超清科技股份有限公司
company entity found by exact match: 现任安徽超清->安徽超清科技股份有限公司
company entity found by exact match: 安徽超清科技股份->安徽超清科技股份有限公司
company entity found by exact match: 超清科技股份有限公司->安徽超清科技股份有限公司
+------------------------------+--------------------+----------+---------------------+
|           

company entity found: 中国->尚科宁家（中国）科技有限公司, sim = 0.9905265283315191
company entity found: 安徽->安徽爱依特科技有限公司, sim = 0.9904600772049361
company entity found: 科技股份->华立捷科技股份有限公司, sim = 0.9999999984295311
process 1-Gram
university entity found: 安徽财经大学->安徽财经大学, sim = 1.0
company entity found by exact match: 安徽超清->安徽超清科技股份有限公司
company entity found by exact match: 超清科技股份->安徽超清科技股份有限公司
process 2-Gram
company entity found by exact match: 安徽超清科技股份->安徽超清科技股份有限公司
company entity found by exact match: 超清科技股份有限公司->安徽超清科技股份有限公司
+---------+--------------------+----------+---------------------+
| Company | Company_Similarity | Position | Position_Similarity |
+---------+--------------------+----------+---------------------+
+---------+--------------------+----------+---------------------+
+--------------+-------------------+--------+-------------------+
|    School    | School_Similarity | Degree | Degree_Similarity |
+--------------+-------------------+--------+-------------------+
| 安徽财经大学 |        1.0   

company entity found: 中国->尚科宁家（中国）科技有限公司, sim = 0.9905265283315191
company entity found: 安徽->安徽爱依特科技有限公司, sim = 0.9904600772049361
degree entity found: 研究生->研究生, sim = 1.0
position entity found: 总经理->总经理, sim = 1.0000000596046448
position entity found: 副董事长->副董事长, sim = 1.0
company entity found: 科技->多玛凯拔科技有限公司, sim = 1.0000000094492707
position entity found: 董事->董事, sim = 1.0
position entity found: 董事长->董事长, sim = 1.000000029802326
process 1-Gram
position entity found: 执行董事->执行董事, sim = 1.000000029802326
company entity found by exact match: 安徽戈瑞->安徽戈瑞电子科技股份有限公司
company entity found by exact match: 戈瑞电子科技->安徽戈瑞电子科技股份有限公司
process 2-Gram
company entity found by exact match: 安徽戈瑞电子科技->安徽戈瑞电子科技股份有限公司
company entity found by exact match: 戈瑞电子科技有限公司->安徽戈瑞电子科技股份有限公司
+------------------------------+--------------------+----------+---------------------+
|           Company            | Company_Similarity | Position | Position_Similarity |
+------------------------------+--------------------+----